In [1]:

import json
import os
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Seq2SeqTrainer, Seq2SeqTrainingArguments, T5ForConditionalGeneration,
    T5Tokenizer, TrainerCallback
)
import matplotlib.pyplot as plt
from functools import partial


In [2]:

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_loss = []
        self.eval_loss = []
        self.training_accuracy = []
        self.eval_accuracy = []
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            if 'loss' in logs:
                self.training_loss.append((state.global_step, logs['loss']))
            if 'eval_loss' in logs:
                self.eval_loss.append((state.global_step, logs['eval_loss']))
            if 'accuracy' in logs:
                self.training_accuracy.append((state.global_step, logs['accuracy']))
            if 'eval_accuracy' in logs:
                self.eval_accuracy.append((state.global_step, logs['eval_accuracy']))


In [3]:

def plot_metrics(callback):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 1, 1)
    if callback.training_loss:
        steps, losses = zip(*callback.training_loss)
        plt.plot(steps, losses, label='Training Loss')
    if callback.eval_loss:
        steps, losses = zip(*callback.eval_loss)
        plt.plot(steps, losses, label='Validation Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    if callback.training_accuracy:
        steps, acc = zip(*callback.training_accuracy)
        plt.plot(steps, acc, label='Training Accuracy')
    if callback.eval_accuracy:
        steps, acc = zip(*callback.eval_accuracy)
        plt.plot(steps, acc, label='Validation Accuracy')
    plt.xlabel('Steps')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.close()

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/t5-small/bd944e5f1b3ad9b70dd9d00010a517059e19265671076b8b0a4a58d9491842bc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1733061408&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzA2MTQwOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby90NS1zbWFsbC9iZDk0NGU1ZjFiM2FkOWI3MGRkOWQwMDAxMGE1MTcwNTllMTkyNjU2NzEwNzZiOGIwYTRhNThkOTQ5MTg0MmJjP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=RA%7E9%7EQBEA4WVnNcgkZioCr1mf-NYNKBU3jLrjBeC42o8mKRDbV5H0FxOGvpIO-Qsoa0BzY01%7EaTX5xNzitZ%7ElKwL1ZroeZ-XoN5ETGoaO1pEDBu2Mtm8fmkW8xOg%7E3kbn9WAgV8SyWO80B4hV0Q1L%7ERpXgFWkiYvzPVqFJYBK9rtZ1nyUAZiOk2WjQ1ic5cktUeyshr3YoKdysA4sW7Vdi4d%7E%7EkQLpHKzd32wfxuHz8Ceo7PjDVpaUsPvmrXdxQOlktxgN9bJjGT2WVMCP1LvapmIH9pMz3P45DHbFX1LZ5wCUC%7EwV45eXQKQtY0Gcqq-DrFu6rwuggUjdlcZBf8hQ__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn-lfs.hf.c

model.safetensors:  48%|####7     | 115M/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
def preprocess_function(examples, tokenizer):
    model_inputs = tokenizer(examples['inputs'], truncation=True, padding=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['labels'], truncation=True, padding=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function_with_tokenizer = partial(preprocess_function, tokenizer=tokenizer)

In [5]:

# Load and prepare data
with open('label_subset.json', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

input_lines = []
label_lines = []
for entry in data:
    input_lines.append(entry['Content'])
    aspects = [term for aspect in entry['Aspects'] for term in aspect['AspectTerms']]
    label_lines.append(' '.join(aspects))

dataset = Dataset.from_dict({'inputs': input_lines, 'labels': label_lines})
tokenized_datasets = dataset.map(
    preprocess_function_with_tokenizer,
    batched=True,
    remove_columns=['inputs'],
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/1100 [00:00<?, ? examples/s]

In [6]:

# Split dataset into train and validation
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(
    "./",
    do_train=True,
    do_eval=True,  # Enable evaluation
    evaluation_strategy="epoch",  # Evaluate after each epoch
    num_train_epochs=30,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir='./',
    group_by_length=True,
    save_strategy="epoch",
    save_total_limit=3,
    fp16=True,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Add callbacks
metrics_callback = MetricsCallback()
trainer.add_callback(metrics_callback)

c:\Users\kietd\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:


# Train the model
trainer.train()

# Plot metrics
plot_metrics(metrics_callback)

# Save the model
save_directory = "FlexiModel"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

  0%|          | 0/26400 [00:00<?, ?it/s]

{'loss': 9.8962, 'grad_norm': 64.56928253173828, 'learning_rate': 3.7878787878787882e-06, 'epoch': 0.57}


  0%|          | 0/220 [00:00<?, ?it/s]

{'eval_loss': 0.886273980140686, 'eval_runtime': 11.8453, 'eval_samples_per_second': 18.573, 'eval_steps_per_second': 18.573, 'epoch': 1.0}
{'loss': 3.183, 'grad_norm': 20.994333267211914, 'learning_rate': 7.5757575757575764e-06, 'epoch': 1.14}
{'loss': 1.1187, 'grad_norm': 8.19991683959961, 'learning_rate': 9.928229665071771e-06, 'epoch': 1.7}


  0%|          | 0/220 [00:00<?, ?it/s]

...

{'eval_loss': 0.16096235811710358, 'eval_runtime': 12.7176, 'eval_samples_per_second': 17.299, 'eval_steps_per_second': 17.299, 'epoch': 30.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'train_runtime': 14057.086, 'train_samples_per_second': 1.878, 'train_steps_per_second': 1.878, 'train_loss': 0.4754251919370709, 'epoch': 30.0}


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


('FlexiModel\\tokenizer_config.json',
 'FlexiModel\\special_tokens_map.json',
 'FlexiModel\\spiece.model',
 'FlexiModel\\added_tokens.json')